In [1]:
import plotly.express as px
import pandas as pd
import os

In [2]:
# Importar as métricas relevantes
metricas_relevantes = [
    # Métricas de tempo de execução
    "elapsedTime",  # Tempo total de execução em ms
    "executorRunTime",  # Tempo total gasto pelos executores
    "executorCpuTime",  # Tempo de CPU utilizado pelos executores
    "jvmGCTime",  # Tempo gasto em coleta de lixo
    # Eficiência de recursos e gargalos
    "shuffleFetchWaitTime",  # Tempo esperando por dados de shuffle
    "shuffleWriteTime",  # Tempo gasto escrevendo dados de shuffle
    # Utilização de memória e spill
    "peakExecutionMemory",  # Pico de memória utilizada
    "diskBytesSpilled",  # Dados despejados em disco quando memória insuficiente
    "memoryBytesSpilled",  # Dados despejados em memória
    # Métricas de shuffle
    "shuffleTotalBytesRead",  # Bytes totais lidos durante shuffle
    "shuffleBytesWritten",  # Bytes escritos durante shuffle
    "shuffleRecordsRead",  # Registros lidos durante shuffle
    "shuffleRecordsWritten",  # Registros escritos durante shuffle
    # Métricas de entrada/saída
    "recordsRead",  # Registros lidos
    "bytesRead",  # Bytes lidos
    "recordsWritten",  # Registros escritos
    "bytesWritten",  # Bytes escritos
    # Paralelismo
    "numStages",  # Número de estágios
    "numTasks",  # Número de tarefas
    "avg_active_tasks",  # Média de tarefas ativas (paralelismo)
]

In [3]:
# Base path
base_path = '/home/guilhermerc/Documentos/workspace/pyspark-playground/output'

# Encontrar todas as pastas de queries
query_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f.startswith('q')]

# Lista para armazenar todos os dataframes
all_dfs = []

# Para cada query, carregar todos os modos de execução disponíveis
for query in query_folders:
    query_path = os.path.join(base_path, query)
    execution_modes = [f for f in os.listdir(query_path) if os.path.isdir(os.path.join(query_path, f))]
    
    for mode in execution_modes:
        mode_path = os.path.join(query_path, mode)
        grouped_file = os.path.join(mode_path, f"{mode}.grouped")
        
        if os.path.exists(grouped_file):
            # Carregar o arquivo
            df = pd.read_csv(grouped_file)
            
            # Adicionar colunas para identificação
            df['execution_mode'] = mode
            df['query'] = query
            
            # Salvar na lista
            all_dfs.append(df)

# Combinar todos os dataframes
df_all_queries = pd.concat(all_dfs, ignore_index=True)

# Mapeamento para nomes mais amigáveis dos modos de execução
execution_mode_mapping = {
    'aqe': 'Com AQE',
    'wo': 'Sem otimizações',
    'cbo': 'Com CBO',
    'cboaqe': 'Com CBO+AQE'
}

# Aplicar o mapeamento
df_all_queries['execution_type'] = df_all_queries['execution_mode'].map(execution_mode_mapping)

df_all_queries = df_all_queries[df_all_queries['query'] != 'q90']  # Remover a query q1, que não possui dados relevantes


In [4]:
# Gráfico de uso de memória
fig_memory = px.bar(
    df_all_queries,
    x='query',
    y='peakExecutionMemory',
    color='execution_type',
    barmode='group',
    title='Uso de Memória por Query e Tipo de Execução',
    labels={'peakExecutionMemory': 'Memória Máxima Utilizada', 'query': 'Query', 'execution_type': 'Tipo de Execução'}
)
fig_memory.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_memory.show()
# Salvar os dados em formato JSON com encoding UTF-8 para evitar caracteres escapados
output_path = os.path.join('agg_results', 'peakExecutionMemory_por_query_por_execution_type.json')
df_all_queries[['query', 'peakExecutionMemory', 'execution_type']].to_json(output_path, orient='records', indent=4, force_ascii=False)


In [5]:
# Gráfico de uso de tempo de CPU
fig_cpu_time = px.bar(
    df_all_queries,
    x='query',
    y='executorCpuTime',
    color='execution_type',
    barmode='group',
    title='Uso de Tempo de CPU por Query e Tipo de Execução',
    labels={'executorCpuTime': 'Tempo de CPU Utilizado', 'query': 'Query', 'execution_type': 'Tipo de Execução'}
)
fig_cpu_time.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_cpu_time.show()
# Salvar os dados em formato JSON com encoding UTF-8 para evitar caracteres escapados
output_path = os.path.join('agg_results', 'executorCpuTime_por_query_por_execution_type.json')
df_all_queries[['query', 'executorCpuTime', 'execution_type']].to_json(output_path, orient='records', indent=4, force_ascii=False)


In [6]:
import json

# Plotar o gráfico
fig_execution_time = px.bar(
    df_all_queries,
    x='query',
    y='elapsedTime',
    color='execution_type',
    barmode='group',
    title='Tempo de Execução por Query e Tipo de Execução',
    labels={'elapsedTime': 'Tempo de Execução (ms)', 'query': 'Query', 'execution_type': 'Tipo de Execução'}
)
fig_execution_time.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_execution_time.show()

# Converter os dados do gráfico para JSON e imprimir
json_data = df_all_queries[['query', 'elapsedTime', 'execution_type']].to_json(orient='records', indent=4)
print("Dados do gráfico em formato JSON:")
print(json_data)

Dados do gráfico em formato JSON:
[
    {
        "query":"q19",
        "elapsedTime":2318,
        "execution_type":"Com CBO"
    },
    {
        "query":"q19",
        "elapsedTime":1908,
        "execution_type":"Com CBO+AQE"
    },
    {
        "query":"q19",
        "elapsedTime":1990,
        "execution_type":"Com AQE"
    },
    {
        "query":"q19",
        "elapsedTime":2751,
        "execution_type":"Sem otimiza\u00e7\u00f5es"
    },
    {
        "query":"q73",
        "elapsedTime":3333,
        "execution_type":"Com CBO"
    },
    {
        "query":"q73",
        "elapsedTime":1972,
        "execution_type":"Com CBO+AQE"
    },
    {
        "query":"q73",
        "elapsedTime":2156,
        "execution_type":"Com AQE"
    },
    {
        "query":"q73",
        "elapsedTime":3233,
        "execution_type":"Sem otimiza\u00e7\u00f5es"
    },
    {
        "query":"q55",
        "elapsedTime":1650,
        "execution_type":"Com CBO"
    },
    {
        "query":"q55",


In [7]:
# Gráfico de operações de shuffle
fig_shuffle = px.bar(
    df_all_queries,
    x='query',
    y=['shuffleBytesWritten', 'shuffleTotalBytesRead'],
    color='execution_type',
    barmode='group',
    title='Operações de Shuffle por Query e Tipo de Execução',
    labels={'value': 'Bytes', 'query': 'Query', 'execution_type': 'Tipo de Execução', 'variable': 'Operação'}
)
fig_shuffle.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_shuffle.show()

# Salvar os dados em formato JSON com encoding UTF-8 para evitar caracteres escapados
# output_path = os.path.join('agg_results', 'shuffle_por_query_por_execution_type.json')
# df_all_queries[['query', 'shuffleBytesWritten', 'shuffleTotalBytesRead', 'execution_type']].to_json(output_path, orient='records', indent=4, force_ascii=False)

In [8]:
# Calcular o speedup
pivot_df = df_all_queries.pivot_table(
    index=['query'], 
    columns='execution_mode', 
    values='elapsedTime'
).reset_index()

# Adicionar colunas de speedup
if 'wo' in pivot_df.columns:
    if 'aqe' in pivot_df.columns:
        pivot_df['speedup_aqe'] = pivot_df['wo'] / pivot_df['aqe']
    if 'cbo' in pivot_df.columns:
        pivot_df['speedup_cbo'] = pivot_df['wo'] / pivot_df['cbo']
    if 'cboaqe' in pivot_df.columns:
        pivot_df['speedup_cboaqe'] = pivot_df['wo'] / pivot_df['cboaqe']

# Transformar para formato long para plotagem
speedup_cols = [col for col in pivot_df.columns if col.startswith('speedup_')]
if speedup_cols:
    speedup_df = pivot_df.melt(
        id_vars=['query'],
        value_vars=speedup_cols,
        var_name='optimization',
        value_name='speedup'
    )
    
    # Melhorar rótulos para o gráfico
    speedup_df['optimization'] = speedup_df['optimization'].str.replace('speedup_', '')
    speedup_df['optimization'] = speedup_df['optimization'].map({
        'aqe': 'AQE',
        'cbo': 'CBO',
        'cboaqe': 'CBO+AQE'
    })
    
    # Plotar o speedup
    fig_speedup = px.bar(
        speedup_df,
        x='query',
        y='speedup',
        color='optimization',
        barmode='group',
        title='Speedup em Relação à Execução Sem Otimizações',
        labels={'speedup': 'Speedup (x vezes)', 'query': 'Query', 'optimization': 'Otimização'}
    )
    fig_speedup.update_layout(xaxis={'categoryorder': 'category ascending'})
    fig_speedup.add_hline(y=1, line_dash="dash", line_color="gray")
    fig_speedup.show()

In [9]:
# Calcular o speedup
pivot_df = df_all_queries.pivot_table(
    index=['query'], 
    columns='execution_mode', 
    values='elapsedTime'
).reset_index()

# Adicionar colunas de speedup
if 'wo' in pivot_df.columns:
    if 'aqe' in pivot_df.columns:
        pivot_df['speedup_aqe'] = pivot_df['wo'] / pivot_df['aqe']
    if 'cbo' in pivot_df.columns:
        pivot_df['speedup_cbo'] = pivot_df['wo'] / pivot_df['cbo']
    if 'cboaqe' in pivot_df.columns:
        pivot_df['speedup_cboaqe'] = pivot_df['wo'] / pivot_df['cboaqe']

# Transformar para formato long para plotagem
speedup_cols = [col for col in pivot_df.columns if col.startswith('speedup_')]
if speedup_cols:
    speedup_df = pivot_df.melt(
        id_vars=['query'],
        value_vars=speedup_cols,
        var_name='optimization',
        value_name='speedup'
    )
    
    # Melhorar rótulos para o gráfico
    speedup_df['optimization'] = speedup_df['optimization'].str.replace('speedup_', '')
    speedup_df['optimization'] = speedup_df['optimization'].map({
        'aqe': 'AQE',
        'cbo': 'CBO',
        'cboaqe': 'CBO+AQE'
    })
    
    # Plotar o speedup
    fig_speedup = px.bar(
        speedup_df,
        x='query',
        y='speedup',
        color='optimization',
        barmode='group',
        title='Speedup em Relação à Execução Sem Otimizações',
        labels={'speedup': 'Speedup (x vezes)', 'query': 'Query', 'optimization': 'Otimização'}
    )
    fig_speedup.update_layout(xaxis={'categoryorder': 'category ascending'})
    fig_speedup.add_hline(y=1, line_dash="dash", line_color="gray")
    fig_speedup.show()

In [10]:
import json

# Lista de métricas para análise
metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory",
    "shuffleTotalBytesRead",
    "shuffleBytesWritten",
    "bytesRead",
    "bytesWritten"
]

# Iterar sobre cada métrica e criar um gráfico de speedup
for metric in metrics:
    # Calcular o speedup para a métrica atual
    pivot_metric_df = df_all_queries.pivot_table(
        index=['query'], 
        columns='execution_mode', 
        values=metric
    ).reset_index()

    # Adicionar colunas de speedup
    if 'wo' in pivot_metric_df.columns:
        if 'aqe' in pivot_metric_df.columns:
            pivot_metric_df['speedup_aqe'] = pivot_metric_df['wo'] / pivot_metric_df['aqe']
        if 'cbo' in pivot_metric_df.columns:
            pivot_metric_df['speedup_cbo'] = pivot_metric_df['wo'] / pivot_metric_df['cbo']
        if 'cboaqe' in pivot_metric_df.columns:
            pivot_metric_df['speedup_cboaqe'] = pivot_metric_df['wo'] / pivot_metric_df['cboaqe']

    # Transformar para formato long para plotagem
    speedup_cols = [col for col in pivot_metric_df.columns if col.startswith('speedup_')]
    if speedup_cols:
        speedup_metric_df = pivot_metric_df.melt(
            id_vars=['query'],
            value_vars=speedup_cols,
            var_name='optimization',
            value_name='speedup'
        )
        
        # Melhorar rótulos para o gráfico
        speedup_metric_df['optimization'] = speedup_metric_df['optimization'].str.replace('speedup_', '')
        speedup_metric_df['optimization'] = speedup_metric_df['optimization'].map({
            'aqe': 'AQE',
            'cbo': 'CBO',
            'cboaqe': 'CBO+AQE'
        })
        
        # Plotar o gráfico de speedup para a métrica atual
        fig_metric_speedup = px.bar(
            speedup_metric_df,
            x='query',
            y='speedup',
            color='optimization',
            barmode='group',
            title=f'Speedup para a Métrica: {metric}',
            labels={'speedup': 'Speedup (x vezes)', 'query': 'Query', 'optimization': 'Otimização'}
        )
        fig_metric_speedup.update_layout(xaxis={'categoryorder': 'category ascending'})
        fig_metric_speedup.add_hline(y=1, line_dash="dash", line_color="gray")
        fig_metric_speedup.show()
        
        # Converter os resultados para JSON e imprimir
        json_result = speedup_metric_df.to_json(orient='records', indent=4)
        print(f"Resultados para a métrica {metric} em formato JSON:")
        print(json_result)

Resultados para a métrica elapsedTime em formato JSON:
[
    {
        "query":"q19",
        "optimization":"AQE",
        "speedup":1.3824120603
    },
    {
        "query":"q27",
        "optimization":"AQE",
        "speedup":1.3723849372
    },
    {
        "query":"q3",
        "optimization":"AQE",
        "speedup":1.7307001795
    },
    {
        "query":"q42",
        "optimization":"AQE",
        "speedup":1.1763858891
    },
    {
        "query":"q43",
        "optimization":"AQE",
        "speedup":1.1738080869
    },
    {
        "query":"q52",
        "optimization":"AQE",
        "speedup":1.5514403292
    },
    {
        "query":"q55",
        "optimization":"AQE",
        "speedup":1.4939662108
    },
    {
        "query":"q67",
        "optimization":"AQE",
        "speedup":1.678328332
    },
    {
        "query":"q73",
        "optimization":"AQE",
        "speedup":1.4995361781
    },
    {
        "query":"q98",
        "optimization":"AQE",
        "spee

Resultados para a métrica executorCpuTime em formato JSON:
[
    {
        "query":"q19",
        "optimization":"AQE",
        "speedup":1.1491636489
    },
    {
        "query":"q27",
        "optimization":"AQE",
        "speedup":1.1896488883
    },
    {
        "query":"q3",
        "optimization":"AQE",
        "speedup":1.3829254727
    },
    {
        "query":"q42",
        "optimization":"AQE",
        "speedup":1.2995022124
    },
    {
        "query":"q43",
        "optimization":"AQE",
        "speedup":1.1410440123
    },
    {
        "query":"q52",
        "optimization":"AQE",
        "speedup":1.3278282411
    },
    {
        "query":"q55",
        "optimization":"AQE",
        "speedup":1.4395727937
    },
    {
        "query":"q67",
        "optimization":"AQE",
        "speedup":1.0591550505
    },
    {
        "query":"q73",
        "optimization":"AQE",
        "speedup":1.2338622708
    },
    {
        "query":"q98",
        "optimization":"AQE",
        

Resultados para a métrica peakExecutionMemory em formato JSON:
[
    {
        "query":"q19",
        "optimization":"AQE",
        "speedup":10.6845933656
    },
    {
        "query":"q27",
        "optimization":"AQE",
        "speedup":14.8967605375
    },
    {
        "query":"q3",
        "optimization":"AQE",
        "speedup":134.3046557678
    },
    {
        "query":"q42",
        "optimization":"AQE",
        "speedup":2.0378607463
    },
    {
        "query":"q43",
        "optimization":"AQE",
        "speedup":20.7348380731
    },
    {
        "query":"q52",
        "optimization":"AQE",
        "speedup":18.1063097027
    },
    {
        "query":"q55",
        "optimization":"AQE",
        "speedup":16.0405210523
    },
    {
        "query":"q67",
        "optimization":"AQE",
        "speedup":1.7986040722
    },
    {
        "query":"q73",
        "optimization":"AQE",
        "speedup":77.8077591329
    },
    {
        "query":"q98",
        "optimization":"AQ

Resultados para a métrica shuffleTotalBytesRead em formato JSON:
[
    {
        "query":"q19",
        "optimization":"AQE",
        "speedup":1.1183897226
    },
    {
        "query":"q27",
        "optimization":"AQE",
        "speedup":1.2873900254
    },
    {
        "query":"q3",
        "optimization":"AQE",
        "speedup":1.1137435937
    },
    {
        "query":"q42",
        "optimization":"AQE",
        "speedup":1.1001535761
    },
    {
        "query":"q43",
        "optimization":"AQE",
        "speedup":1.1004282971
    },
    {
        "query":"q52",
        "optimization":"AQE",
        "speedup":1.1119817344
    },
    {
        "query":"q55",
        "optimization":"AQE",
        "speedup":1.110539506
    },
    {
        "query":"q67",
        "optimization":"AQE",
        "speedup":1.000377774
    },
    {
        "query":"q73",
        "optimization":"AQE",
        "speedup":1.000053403
    },
    {
        "query":"q98",
        "optimization":"AQE",
     

Resultados para a métrica shuffleBytesWritten em formato JSON:
[
    {
        "query":"q19",
        "optimization":"AQE",
        "speedup":1.1183897226
    },
    {
        "query":"q27",
        "optimization":"AQE",
        "speedup":1.2873900254
    },
    {
        "query":"q3",
        "optimization":"AQE",
        "speedup":1.1137435937
    },
    {
        "query":"q42",
        "optimization":"AQE",
        "speedup":1.1001535761
    },
    {
        "query":"q43",
        "optimization":"AQE",
        "speedup":1.1004282971
    },
    {
        "query":"q52",
        "optimization":"AQE",
        "speedup":1.1119817344
    },
    {
        "query":"q55",
        "optimization":"AQE",
        "speedup":1.110539506
    },
    {
        "query":"q67",
        "optimization":"AQE",
        "speedup":1.000377774
    },
    {
        "query":"q73",
        "optimization":"AQE",
        "speedup":1.0000689136
    },
    {
        "query":"q98",
        "optimization":"AQE",
      

Resultados para a métrica bytesRead em formato JSON:
[
    {
        "query":"q19",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q27",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q3",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q42",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q43",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q52",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q55",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q67",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q73",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q98",
        "optimization":"AQE",
        "speedup":1.0
    },
    {
        "query":"q19",
        "optimization":"CBO",
       

Resultados para a métrica bytesWritten em formato JSON:
[
    {
        "query":"q19",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q27",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q3",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q42",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q43",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q52",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q55",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q67",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q73",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q98",
        "optimization":"AQE",
        "speedup":null
    },
    {
        "query":"q19",
        "optimization":"

In [11]:
# Mapeamento do número de estágios por query
stages_mapping = {
    'q42': 5,
    'q43': 5,
    'q3': 5,
    'q52': 5,
    'q55': 5,
    'q27': 7,
    'q67': 7,
    'q19': 8,
    'q98': 8,
    'q73': 9
}

# Adicionar o número de estágios ao dataframe principal
df_all_queries['num_stages'] = df_all_queries['query'].map(stages_mapping)

# Filtrar os dados para remover o baseline "Sem otimizações"
filtered_df = df_all_queries[df_all_queries['execution_mode'] != 'wo']

# Calcular a porcentagem de melhoria ou piora para cada tipo de execução
baseline_elapsed_time = df_all_queries[df_all_queries['execution_mode'] == 'wo'].set_index('query')['elapsedTime']

# Calcular a porcentagem de melhoria ou piora agrupando por número de estágios
baseline_elapsed_time_by_stage = df_all_queries[df_all_queries['execution_mode'] == 'wo'].groupby('num_stages')['elapsedTime'].mean()

filtered_df['elapsedTime_percentage_change'] = filtered_df.apply(
    lambda row: ((baseline_elapsed_time_by_stage[row['num_stages']] - row['elapsedTime']) / baseline_elapsed_time_by_stage[row['num_stages']]) * 100,
    axis=1
)

# Gráfico de porcentagem por query
fig_percentage_by_query = px.bar(
    filtered_df,
    x='query',
    y='elapsedTime_percentage_change',
    color='execution_type',
    barmode='group',
    title='Porcentagem de Melhoria ou Piora no Tempo de Execução por Query',
    labels={'elapsedTime_percentage_change': 'Porcentagem (%)', 'query': 'Query', 'execution_type': 'Tipo de Execução'}
)
fig_percentage_by_query.update_layout(xaxis={'categoryorder': 'category ascending'})
fig_percentage_by_query.show()

# Calcular a média de porcentagem por tipo de execução
average_percentage_change = filtered_df.groupby('execution_type')['elapsedTime_percentage_change'].mean().reset_index()

# Gráfico de média de porcentagem por tipo de execução
fig_average_percentage = px.bar(
    average_percentage_change,
    x='execution_type',
    y='elapsedTime_percentage_change',
    title='Média de Porcentagem de Melhoria ou Piora no Tempo de Execução por Tipo de Execução',
    labels={'elapsedTime_percentage_change': 'Porcentagem Média (%)', 'execution_type': 'Tipo de Execução'},
    text='elapsedTime_percentage_change',
    color='execution_type'
)
fig_average_percentage.update_layout(
    xaxis_title="Tipo de Execução",
    yaxis_title="Porcentagem Média (%)",
    showlegend=False
)
fig_average_percentage.show()


/tmp/ipykernel_37217/877398385.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [13]:
# Dicionário para ajustar os rótulos de acordo com a métrica
metric_labels = {
    "elapsedTime": "Redução no Tempo de Execução",
    "executorCpuTime": "Redução no Tempo de CPU",
    "peakExecutionMemory": "Redução no Uso de Memória",
    "shuffleTotalBytesRead": "Redução nos Bytes Lidos (Shuffle)",
    "shuffleBytesWritten": "Redução nos Bytes Escritos (Shuffle)",
    "bytesRead": "Redução nos Bytes Lidos",
    "bytesWritten": "Redução nos Bytes Escritos"
}

# Iterar sobre cada métrica e criar um gráfico de melhoria agrupado
for metric in metrics:
    # Calcular o speedup para a métrica atual
    pivot_metric_df = df_all_queries.pivot_table(
        index=['query'], 
        columns='execution_mode', 
        values=metric
    ).reset_index()

    # Adicionar colunas de melhoria
    if 'wo' in pivot_metric_df.columns:
        if 'aqe' in pivot_metric_df.columns:
            pivot_metric_df['improvement_aqe'] = pivot_metric_df['wo'] / pivot_metric_df['aqe']
        if 'cbo' in pivot_metric_df.columns:
            pivot_metric_df['improvement_cbo'] = pivot_metric_df['wo'] / pivot_metric_df['cbo']
        if 'cboaqe' in pivot_metric_df.columns:
            pivot_metric_df['improvement_cboaqe'] = pivot_metric_df['wo'] / pivot_metric_df['cboaqe']

    # Transformar para formato long para cálculo da média
    improvement_cols = [col for col in pivot_metric_df.columns if col.startswith('improvement_')]
    if improvement_cols:
        improvement_metric_df = pivot_metric_df.melt(
            id_vars=['query'],
            value_vars=improvement_cols,
            var_name='optimization',
            value_name='improvement'
        )
        
        # Melhorar rótulos para o gráfico
        improvement_metric_df['optimization'] = improvement_metric_df['optimization'].str.replace('improvement_', '')
        improvement_metric_df['optimization'] = improvement_metric_df['optimization'].map({
            'aqe': 'AQE',
            'cbo': 'CBO',
            'cboaqe': 'CBO+AQE'
        })

        improvement_metric_df = improvement_metric_df[improvement_metric_df['query'] != 'q27']  # Remover a query q1, que não possui dados relevantes
        
        # Calcular a média de melhoria por tipo de otimização
        avg_improvement_metric = improvement_metric_df.groupby('optimization')['improvement'].mean().reset_index()
        
        # Obter o rótulo da métrica
        metric_label = metric_labels.get(metric, metric)
        
        # Plotar o gráfico de média de melhoria para a métrica atual
        fig_avg_metric = px.bar(
            avg_improvement_metric,
            x='optimization',
            y='improvement',
            title=f'Média de {metric_label}',
            labels={'improvement': f'{metric_label} (x vezes)', 'optimization': 'Otimização'},
            text='improvement',
            color='optimization'
        )
        
        # Melhorar o layout
        fig_avg_metric.update_layout(
            xaxis_title="Otimização",
            yaxis_title=f'{metric_label} (x vezes)',
            showlegend=False
        )
        
        # Adicionar linha de referência no valor 1
        fig_avg_metric.add_hline(y=1, line_dash="dash", line_color="gray")
        
        # Mostrar o gráfico
        fig_avg_metric.show()
        
        # Converter os dados para JSON e imprimir
        json_result = avg_improvement_metric.to_json(orient='records', indent=4)
        print(f"Resultados para a métrica {metric} em formato JSON:")
        print(json_result)


Resultados para a métrica elapsedTime em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.5092116173
    },
    {
        "optimization":"CBO",
        "improvement":1.0950668013
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.6321194732
    }
]


Resultados para a métrica executorCpuTime em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.409331031
    },
    {
        "optimization":"CBO",
        "improvement":1.0207080099
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.4180101545
    }
]


Resultados para a métrica peakExecutionMemory em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":33.7923222327
    },
    {
        "optimization":"CBO",
        "improvement":1.0
    },
    {
        "optimization":"CBO+AQE",
        "improvement":33.7923222327
    }
]


Resultados para a métrica shuffleTotalBytesRead em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.1064757438
    },
    {
        "optimization":"CBO",
        "improvement":1.0000015929
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.1064702973
    }
]


Resultados para a métrica shuffleBytesWritten em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.0811045514
    },
    {
        "optimization":"CBO",
        "improvement":1.0000016376
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.081104079
    }
]


Resultados para a métrica bytesRead em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.0
    },
    {
        "optimization":"CBO",
        "improvement":1.0
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.0
    }
]


Resultados para a métrica bytesWritten em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":null
    },
    {
        "optimization":"CBO",
        "improvement":null
    },
    {
        "optimization":"CBO+AQE",
        "improvement":null
    }
]


In [13]:
# Dicionário para ajustar os rótulos de acordo com a métrica
metric_labels = {
    "elapsedTime": "Redução no Tempo de Execução",
    "executorCpuTime": "Redução no Tempo de CPU",
    "peakExecutionMemory": "Redução no Uso de Memória",
    "shuffleTotalBytesRead": "Redução nos Bytes Lidos (Shuffle)",
    "shuffleBytesWritten": "Redução nos Bytes Escritos (Shuffle)",
    "bytesRead": "Redução nos Bytes Lidos",
    "bytesWritten": "Redução nos Bytes Escritos"
}

# Iterar sobre cada métrica e criar um gráfico de melhoria agrupado
for metric in metrics:
    # Calcular o speedup para a métrica atual
    pivot_metric_df = df_all_queries.pivot_table(
        index=['query'], 
        columns='execution_mode', 
        values=metric
    ).reset_index()

    # Adicionar colunas de melhoria
    if 'wo' in pivot_metric_df.columns:
        if 'aqe' in pivot_metric_df.columns:
            pivot_metric_df['improvement_aqe'] = pivot_metric_df['wo'] / pivot_metric_df['aqe']
        if 'cbo' in pivot_metric_df.columns:
            pivot_metric_df['improvement_cbo'] = pivot_metric_df['wo'] / pivot_metric_df['cbo']
        if 'cboaqe' in pivot_metric_df.columns:
            pivot_metric_df['improvement_cboaqe'] = pivot_metric_df['wo'] / pivot_metric_df['cboaqe']

    # Transformar para formato long para cálculo da média
    improvement_cols = [col for col in pivot_metric_df.columns if col.startswith('improvement_')]
    if improvement_cols:
        improvement_metric_df = pivot_metric_df.melt(
            id_vars=['query'],
            value_vars=improvement_cols,
            var_name='optimization',
            value_name='improvement'
        )
        
        # Melhorar rótulos para o gráfico
        improvement_metric_df['optimization'] = improvement_metric_df['optimization'].str.replace('improvement_', '')
        improvement_metric_df['optimization'] = improvement_metric_df['optimization'].map({
            'aqe': 'AQE',
            'cbo': 'CBO',
            'cboaqe': 'CBO+AQE'
        })
        
        # Criar uma cópia dos dados CBO+AQE excluindo a query 27
        cboaqe_without_q27 = improvement_metric_df[
            (improvement_metric_df['optimization'] == 'CBO+AQE') & 
            (improvement_metric_df['query'] != 'q27')
        ].copy()
        cboaqe_without_q27['optimization'] = 'CBO+AQE (sem q27)'
        
        # Combinar os dataframes originais com o novo
        improvement_metric_df = pd.concat([improvement_metric_df, cboaqe_without_q27], ignore_index=True)
        
        # Calcular a média de melhoria por tipo de otimização
        avg_improvement_metric = improvement_metric_df.groupby('optimization')['improvement'].mean().reset_index()
        
        # Obter o rótulo da métrica
        metric_label = metric_labels.get(metric, metric)
        
        # Plotar o gráfico de média de melhoria para a métrica atual
        fig_avg_metric = px.bar(
            avg_improvement_metric,
            x='optimization',
            y='improvement',
            title=f'Média de {metric_label}',
            labels={'improvement': f'{metric_label} (x vezes)', 'optimization': 'Otimização'},
            text='improvement',
            color='optimization'
        )
        
        # Melhorar o layout
        fig_avg_metric.update_layout(
            xaxis_title="Otimização",
            yaxis_title=f'{metric_label} (x vezes)',
            showlegend=False
        )
        
        # Adicionar linha de referência no valor 1
        fig_avg_metric.add_hline(y=1, line_dash="dash", line_color="gray")
        
        # Mostrar o gráfico
        fig_avg_metric.show()
        
        # Converter os dados para JSON e imprimir
        json_result = avg_improvement_metric.to_json(orient='records', indent=4)
        print(f"Resultados para a métrica {metric} em formato JSON:")
        print(json_result)


Resultados para a métrica elapsedTime em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.4955289493
    },
    {
        "optimization":"CBO",
        "improvement":1.0015105194
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.4940560787
    },
    {
        "optimization":"CBO+AQE (sem q27)",
        "improvement":1.6321194732
    }
]


Resultados para a métrica executorCpuTime em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.3873628167
    },
    {
        "optimization":"CBO",
        "improvement":0.9356209917
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.3033879999
    },
    {
        "optimization":"CBO+AQE (sem q27)",
        "improvement":1.4180101545
    }
]


Resultados para a métrica peakExecutionMemory em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":31.9027660632
    },
    {
        "optimization":"CBO",
        "improvement":0.9242598968
    },
    {
        "optimization":"CBO+AQE",
        "improvement":31.9027660632
    },
    {
        "optimization":"CBO+AQE (sem q27)",
        "improvement":33.7923222327
    }
]


Resultados para a métrica shuffleTotalBytesRead em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.1245671719
    },
    {
        "optimization":"CBO",
        "improvement":0.9004148215
    },
    {
        "optimization":"CBO+AQE",
        "improvement":0.996238536
    },
    {
        "optimization":"CBO+AQE (sem q27)",
        "improvement":1.1064702973
    }
]


Resultados para a métrica shuffleBytesWritten em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.1017330988
    },
    {
        "optimization":"CBO",
        "improvement":0.9004148618
    },
    {
        "optimization":"CBO+AQE",
        "improvement":0.9734089396
    },
    {
        "optimization":"CBO+AQE (sem q27)",
        "improvement":1.081104079
    }
]


Resultados para a métrica bytesRead em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":1.0
    },
    {
        "optimization":"CBO",
        "improvement":1.0
    },
    {
        "optimization":"CBO+AQE",
        "improvement":1.0
    },
    {
        "optimization":"CBO+AQE (sem q27)",
        "improvement":1.0
    }
]


Resultados para a métrica bytesWritten em formato JSON:
[
    {
        "optimization":"AQE",
        "improvement":null
    },
    {
        "optimization":"CBO",
        "improvement":null
    },
    {
        "optimization":"CBO+AQE",
        "improvement":null
    },
    {
        "optimization":"CBO+AQE (sem q27)",
        "improvement":null
    }
]


In [ ]:
# Filtrar os dados para considerar apenas a query 27
query_27_speedup_df = speedup_df[speedup_df['query'] != 'q27']

# Calcular a média de speedup por tipo de otimização para a query 27
average_speedup_query_27 = query_27_speedup_df.groupby('optimization')['speedup'].mean().reset_index()

# Plotar o gráfico de barras com a média de speedup para a query 27
fig_avg_speedup_query_27 = px.bar(
    average_speedup_query_27,
    x='optimization',
    y='speedup',
    title='Média de Speedup por Tipo de Otimização',
    labels={'speedup': 'Speedup Médio (x vezes)', 'optimization': 'Otimização'},
    text='speedup',
    color='optimization'
)

# Melhorar o layout
fig_avg_speedup_query_27.update_layout(
    xaxis_title="Otimização",
    yaxis_title="Speedup Médio (x vezes)",
    showlegend=False
)

# Adicionar linha de referência no valor 1
fig_avg_speedup_query_27.add_hline(y=1, line_dash="dash", line_color="gray")

# Mostrar o gráfico
fig_avg_speedup_query_27.show()


In [23]:
# Lista de métricas para análise
metrics_to_plot = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory",
    "shuffleTotalBytesRead",
    "shuffleBytesWritten",
    "bytesRead",
    "bytesWritten"
]

# Iterar sobre cada métrica e criar um gráfico
for metric in metrics_to_plot:
    fig = px.bar(
        df_all_queries,
        x='query',
        y=metric,
        color='execution_type',
        barmode='group',
        title=f'Análise da Métrica: {metric}',
        labels={metric: metric, 'query': 'Query', 'execution_type': 'Tipo de Execução'}
    )
    fig.update_layout(xaxis={'categoryorder': 'category ascending'})
    fig.show()

In [11]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

# Selecionar as métricas para o gráfico de radar
radar_metrics = ['elapsedTime', 'executorRunTime', 'executorCpuTime', 
                 'shuffleTotalBytesRead', 'shuffleBytesWritten', 'peakExecutionMemory']

# Para cada query, criar um gráfico de radar
for query in df_all_queries['query'].unique():
    query_data = df_all_queries[df_all_queries['query'] == query]
    
    # Normalizar os dados para a escala 0-1
    scaler = MinMaxScaler()
    query_data_norm = pd.DataFrame(
        scaler.fit_transform(query_data[radar_metrics]), 
        columns=radar_metrics
    )
    query_data_norm['execution_type'] = query_data['execution_type'].values
    
    # Criar gráfico de radar
    fig = go.Figure()
    
    for exec_type in query_data_norm['execution_type'].unique():
        exec_data = query_data_norm[query_data_norm['execution_type'] == exec_type]
        
        fig.add_trace(go.Scatterpolar(
            r=exec_data[radar_metrics].values[0],
            theta=radar_metrics,
            fill='toself',
            name=exec_type
        ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )
        ),
        title=f'Comparação de Métricas para Query {query}',
        showlegend=True
    )
    fig.show()

In [16]:
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler

# Selecionar as métricas para o gráfico de radar
radar_metrics = ['elapsedTime', 'executorRunTime', 'executorCpuTime', 
                 'shuffleTotalBytesRead', 'shuffleBytesWritten', 'peakExecutionMemory']

# Calcular a média das métricas por tipo de execução
mean_data = df_all_queries.groupby('execution_type')[radar_metrics].mean().reset_index()

# Normalizar os dados para a escala 0-1
scaler = MinMaxScaler()
mean_data_norm = pd.DataFrame(
    scaler.fit_transform(mean_data[radar_metrics]), 
    columns=radar_metrics
)
mean_data_norm['execution_type'] = mean_data['execution_type']

# Criar o gráfico de radar
fig = go.Figure()

for exec_type in mean_data_norm['execution_type'].unique():
    exec_data = mean_data_norm[mean_data_norm['execution_type'] == exec_type]
    
    fig.add_trace(go.Scatterpolar(
        r=exec_data[radar_metrics].values[0],
        theta=radar_metrics,
        fill='toself',
        name=exec_type
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    title='Comparação Agregada de Métricas por Tipo de Execução',
    showlegend=True
)

fig.show()

In [2]:
# Calcular throughput de I/O (bytes por milissegundo) para todas as queries
import plotly.express as px
import pandas as pd
import os

# Reutilizar seu código para carregar todos os dados
base_path = '/home/guilhermerc/Documentos/workspace/pyspark-playground/output'

# Encontrar todas as pastas de queries
query_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f.startswith('q')]

# Lista para armazenar todos os dataframes
all_dfs = []

# Para cada query, carregar todos os modos de execução disponíveis
for query in query_folders:
    query_path = os.path.join(base_path, query)
    execution_modes = [f for f in os.listdir(query_path) if os.path.isdir(os.path.join(query_path, f))]
    
    for mode in execution_modes:
        mode_path = os.path.join(query_path, mode)
        grouped_file = os.path.join(mode_path, f"{mode}.grouped")
        
        if os.path.exists(grouped_file):
            # Carregar o arquivo
            df = pd.read_csv(grouped_file)
            
            # Adicionar colunas para identificação
            df['execution_mode'] = mode
            df['query'] = query
            
            # Salvar na lista
            all_dfs.append(df)

# Combinar todos os dataframes
df_all_queries = pd.concat(all_dfs, ignore_index=True)

# Mapeamento para nomes mais amigáveis dos modos de execução
execution_mode_mapping = {
    'aqe': 'Com AQE',
    'wo': 'Sem otimizações',
    'cbo': 'Com CBO',
    'cboaqe': 'Com CBO+AQE'
}

# Aplicar o mapeamento
df_all_queries['execution_type'] = df_all_queries['execution_mode'].map(execution_mode_mapping)

df_all_queries = df_all_queries.where(df_all_queries['execution_mode'] == 'wo')

# Calcular o throughput de I/O (bytes por milissegundo)
df_all_queries['io_throughput'] = (df_all_queries['bytesRead'] + df_all_queries['bytesWritten']) / df_all_queries['elapsedTime']

# Criar o gráfico de dispersão
fig = px.scatter(
    df_all_queries,
    x='executorCpuTime',
    y='io_throughput',
    color='execution_type',
    hover_data=['query', 'bytesRead', 'bytesWritten', 'elapsedTime'],
    labels={
        'executorCpuTime': 'Tempo de CPU (ms)',
        'io_throughput': 'Throughput de I/O (bytes/ms)',
        'execution_type': 'Tipo de Execução'
    },
    title='Relação entre Uso de CPU e Throughput de I/O por Query',
    size_max=15,  # Tamanho máximo do ponto
    opacity=0.7   # Transparência para melhor visualização
)

# Ajustes no layout
fig.update_layout(
    xaxis=dict(title='Tempo de CPU (ms)'),
    yaxis=dict(title='Throughput de I/O (bytes/ms)'),
    legend_title="Tipo de Execução",
    hovermode='closest'
)

# Adicionar rótulos para os pontos
fig.update_traces(
    marker=dict(size=10),
    selector=dict(mode='markers')
)

# Mostrar o gráfico
fig.show()

In [ ]:
# Filter data for query Q73
q73_df = df_all_queries[df_all_queries['query'] == 'q73']

# Get the baseline (wo) values
baseline_q73 = q73_df[q73_df['execution_mode'] == 'wo'].iloc[0]

# Select the metrics to analyze
q73_metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory", 
    "shuffleTotalBytesRead",
    "shuffleBytesWritten"
]

# Create a dataframe to store improvement ratios
improvements = []

# Calculate improvements for each configuration compared to baseline
for _, row in q73_df[q73_df['execution_mode'] != 'wo'].iterrows():
    for metric in q73_metrics:
        # Calculate improvement ratio (baseline / configuration)
        improvement = baseline_q73[metric] / row[metric] if row[metric] > 0 else 0
        improvements.append({
            'optimization': row['execution_type'],
            'metric': metric,
            'improvement_ratio': improvement
        })

# Convert to DataFrame
improvements_df = pd.DataFrame(improvements)

# Print the improvement values for memory usage
memory_improvements = improvements_df[improvements_df['metric'] == 'peakExecutionMemory']
print("Memory improvement ratios:")
for _, row in memory_improvements.iterrows():
    print(f"{row['optimization']}: {row['improvement_ratio']:.2f}x")

# Define better labels for metrics
metric_labels = {
    "elapsedTime": "Tempo de Execução",
    "executorCpuTime": "Tempo de CPU",
    "peakExecutionMemory": "Uso de Memória",
    "shuffleTotalBytesRead": "Dados de Shuffle Lidos",
    "shuffleBytesWritten": "Dados de Shuffle Escritos"
}

# Map metrics to better labels
improvements_df['metric_label'] = improvements_df['metric'].map(metric_labels)

# Order the metrics in a logical sequence
metric_order = ["Tempo de Execução", "Tempo de CPU", "Uso de Memória", 
                "Dados de Shuffle Lidos", "Dados de Shuffle Escritos"]
improvements_df['metric_order'] = improvements_df['metric_label'].apply(lambda x: metric_order.index(x))
improvements_df = improvements_df.sort_values('metric_order')

# Define a color map for optimization types to ensure consistency
color_map = {
    'Com CBO': '#636efa',
    'Com AQE': '#00cc96',
    'Com CBO+AQE': '#ef553b'
}

# Create the visualization
fig = px.bar(
    improvements_df,
    x='metric_label',
    y='improvement_ratio',
    color='optimization',
    barmode='group',
    text=improvements_df['improvement_ratio'].round(2),  # Add text labels on bars
    title='Ganhos de Desempenho para Q73 sob Diferentes Otimizações',
    labels={
        'improvement_ratio': 'Fator de Melhoria (x vezes)',
        'metric_label': 'Métrica',
        'optimization': 'Otimização'
    },
    category_orders={"metric_label": metric_order},  # Ensure consistent order
    color_discrete_map=color_map  # Use consistent colors
)

# Add a reference line at y=1 (no improvement)
fig.add_hline(y=1, line_dash="dash", line_color="gray")

# Improve layout
fig.update_layout(
    xaxis_title="Métrica",
    yaxis_title="Fator de Melhoria (x vezes)",
    legend_title="Otimização"
)

# Update text position
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

Memory improvement ratios:
Com CBO: 1.00x
Com CBO+AQE: 77.81x
Com AQE: 77.81x


In [18]:
# Filter data for query Q98
q98_df = df_all_queries[df_all_queries['query'] == 'q98']

# Get the baseline (wo) values
baseline_q98 = q98_df[q98_df['execution_mode'] == 'wo'].iloc[0]

# Select the metrics to analyze
q98_metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory", 
    "shuffleTotalBytesRead",
    "shuffleBytesWritten"
]

# Create a dataframe to store improvement ratios
improvements = []

# Calculate improvements for each configuration compared to baseline
for _, row in q98_df[q98_df['execution_mode'] != 'wo'].iterrows():
    for metric in q98_metrics:
        # Calculate improvement ratio (baseline / configuration)
        improvement = baseline_q98[metric] / row[metric] if row[metric] > 0 else 0
        improvements.append({
            'optimization': row['execution_type'],
            'metric': metric,
            'improvement_ratio': improvement
        })

# Convert to DataFrame
improvements_df = pd.DataFrame(improvements)

# Print the improvement values for memory usage
memory_improvements = improvements_df[improvements_df['metric'] == 'peakExecutionMemory']
print("Memory improvement ratios:")
for _, row in memory_improvements.iterrows():
    print(f"{row['optimization']}: {row['improvement_ratio']:.2f}x")

# Define better labels for metrics
metric_labels = {
    "elapsedTime": "Tempo de Execução",
    "executorCpuTime": "Tempo de CPU",
    "peakExecutionMemory": "Uso de Memória",
    "shuffleTotalBytesRead": "Dados de Shuffle Lidos",
    "shuffleBytesWritten": "Dados de Shuffle Escritos"
}

# Map metrics to better labels
improvements_df['metric_label'] = improvements_df['metric'].map(metric_labels)

# Order the metrics in a logical sequence
metric_order = ["Tempo de Execução", "Tempo de CPU", "Uso de Memória", 
                "Dados de Shuffle Lidos", "Dados de Shuffle Escritos"]
improvements_df['metric_order'] = improvements_df['metric_label'].apply(lambda x: metric_order.index(x))
improvements_df = improvements_df.sort_values('metric_order')

# Define a color map for optimization types to ensure consistency
color_map = {
    'Com CBO': '#636efa',
    'Com AQE': '#00cc96',
    'Com CBO+AQE': '#ef553b'
}

# Create the visualization
fig = px.bar(
    improvements_df,
    x='metric_label',
    y='improvement_ratio',
    color='optimization',
    barmode='group',
    text=improvements_df['improvement_ratio'].round(2),  # Add text labels on bars
    title='Ganhos de Desempenho para Q98 sob Diferentes Otimizações',
    labels={
        'improvement_ratio': 'Fator de Melhoria (x vezes)',
        'metric_label': 'Métrica',
        'optimization': 'Otimização'
    },
    category_orders={"metric_label": metric_order},  # Ensure consistent order
    color_discrete_map=color_map  # Use consistent colors
)

# Add a reference line at y=1 (no improvement)
fig.add_hline(y=1, line_dash="dash", line_color="gray")

# Improve layout
fig.update_layout(
    xaxis_title="Métrica",
    yaxis_title="Fator de Melhoria (x vezes)",
    legend_title="Otimização"
)

# Update text position
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

Memory improvement ratios:
Com CBO: 1.00x
Com CBO+AQE: 22.62x
Com AQE: 22.62x


In [19]:
# Filter data for query Q67
q67_df = df_all_queries[df_all_queries['query'] == 'q67']

# Get the baseline (wo) values
baseline_q67 = q67_df[q67_df['execution_mode'] == 'wo'].iloc[0]

# Select the metrics to analyze
q67_metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory", 
    "shuffleTotalBytesRead",
    "shuffleBytesWritten"
]

# Create a dataframe to store improvement ratios
improvements = []

# Calculate improvements for each configuration compared to baseline
for _, row in q67_df[q67_df['execution_mode'] != 'wo'].iterrows():
    for metric in q67_metrics:
        # Calculate improvement ratio (baseline / configuration)
        improvement = baseline_q67[metric] / row[metric] if row[metric] > 0 else 0
        improvements.append({
            'optimization': row['execution_type'],
            'metric': metric,
            'improvement_ratio': improvement
        })

# Convert to DataFrame
improvements_df = pd.DataFrame(improvements)

# Print the improvement values for memory usage
memory_improvements = improvements_df[improvements_df['metric'] == 'peakExecutionMemory']
print("Memory improvement ratios:")
for _, row in memory_improvements.iterrows():
    print(f"{row['optimization']}: {row['improvement_ratio']:.2f}x")

# Define better labels for metrics
metric_labels = {
    "elapsedTime": "Tempo de Execução",
    "executorCpuTime": "Tempo de CPU",
    "peakExecutionMemory": "Uso de Memória",
    "shuffleTotalBytesRead": "Dados de Shuffle Lidos",
    "shuffleBytesWritten": "Dados de Shuffle Escritos"
}

# Map metrics to better labels
improvements_df['metric_label'] = improvements_df['metric'].map(metric_labels)

# Order the metrics in a logical sequence
metric_order = ["Tempo de Execução", "Tempo de CPU", "Uso de Memória", 
                "Dados de Shuffle Lidos", "Dados de Shuffle Escritos"]
improvements_df['metric_order'] = improvements_df['metric_label'].apply(lambda x: metric_order.index(x))
improvements_df = improvements_df.sort_values('metric_order')

# Define a color map for optimization types to ensure consistency
color_map = {
    'Com CBO': '#636efa',
    'Com AQE': '#00cc96',
    'Com CBO+AQE': '#ef553b'
}

# Create the visualization
fig = px.bar(
    improvements_df,
    x='metric_label',
    y='improvement_ratio',
    color='optimization',
    barmode='group',
    text=improvements_df['improvement_ratio'].round(2),  # Add text labels on bars
    title='Ganhos de Desempenho para Q67 sob Diferentes Otimizações',
    labels={
        'improvement_ratio': 'Fator de Melhoria (x vezes)',
        'metric_label': 'Métrica',
        'optimization': 'Otimização'
    },
    category_orders={"metric_label": metric_order},  # Ensure consistent order
    color_discrete_map=color_map  # Use consistent colors
)

# Add a reference line at y=1 (no improvement)
fig.add_hline(y=1, line_dash="dash", line_color="gray")

# Improve layout
fig.update_layout(
    xaxis_title="Métrica",
    yaxis_title="Fator de Melhoria (x vezes)",
    legend_title="Otimização"
)

# Update text position
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

Memory improvement ratios:
Com CBO: 1.00x
Com CBO+AQE: 1.80x
Com AQE: 1.80x


In [20]:
# Filter data for query Q27
q27_df = df_all_queries[df_all_queries['query'] == 'q27']

# Get the baseline (wo) values
baseline_q27 = q27_df[q27_df['execution_mode'] == 'wo'].iloc[0]

# Select the metrics to analyze
q27_metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory", 
    "shuffleTotalBytesRead",
    "shuffleBytesWritten"
]

# Create a dataframe to store improvement ratios
improvements = []

# Calculate improvements for each configuration compared to baseline
for _, row in q27_df[q27_df['execution_mode'] != 'wo'].iterrows():
    for metric in q27_metrics:
        # Calculate improvement ratio (baseline / configuration)
        improvement = baseline_q27[metric] / row[metric] if row[metric] > 0 else 0
        improvements.append({
            'optimization': row['execution_type'],
            'metric': metric,
            'improvement_ratio': improvement
        })

# Convert to DataFrame
improvements_df = pd.DataFrame(improvements)

# Print the improvement values for memory usage
memory_improvements = improvements_df[improvements_df['metric'] == 'peakExecutionMemory']
print("Memory improvement ratios:")
for _, row in memory_improvements.iterrows():
    print(f"{row['optimization']}: {row['improvement_ratio']:.2f}x")

# Define better labels for metrics
metric_labels = {
    "elapsedTime": "Tempo de Execução",
    "executorCpuTime": "Tempo de CPU",
    "peakExecutionMemory": "Uso de Memória",
    "shuffleTotalBytesRead": "Dados de Shuffle Lidos",
    "shuffleBytesWritten": "Dados de Shuffle Escritos"
}

# Map metrics to better labels
improvements_df['metric_label'] = improvements_df['metric'].map(metric_labels)

# Order the metrics in a logical sequence
metric_order = ["Tempo de Execução", "Tempo de CPU", "Uso de Memória", 
                "Dados de Shuffle Lidos", "Dados de Shuffle Escritos"]
improvements_df['metric_order'] = improvements_df['metric_label'].apply(lambda x: metric_order.index(x))
improvements_df = improvements_df.sort_values('metric_order')

# Define a color map for optimization types to ensure consistency
color_map = {
    'Com CBO': '#636efa',
    'Com AQE': '#00cc96',
    'Com CBO+AQE': '#ef553b'
}

# Create the visualization
fig = px.bar(
    improvements_df,
    x='metric_label',
    y='improvement_ratio',
    color='optimization',
    barmode='group',
    text=improvements_df['improvement_ratio'].round(2),  # Add text labels on bars
    title='Ganhos de Desempenho para Q27 sob Diferentes Otimizações',
    labels={
        'improvement_ratio': 'Fator de Melhoria (x vezes)',
        'metric_label': 'Métrica',
        'optimization': 'Otimização'
    },
    category_orders={"metric_label": metric_order},  # Ensure consistent order
    color_discrete_map=color_map  # Use consistent colors
)

# Add a reference line at y=1 (no improvement)
fig.add_hline(y=1, line_dash="dash", line_color="gray")

# Improve layout
fig.update_layout(
    xaxis_title="Métrica",
    yaxis_title="Fator de Melhoria (x vezes)",
    legend_title="Otimização"
)

# Update text position
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

Memory improvement ratios:
Com CBO: 0.24x
Com CBO+AQE: 14.90x
Com AQE: 14.90x


In [21]:
# Filter data for query Q3
q3_df = df_all_queries[df_all_queries['query'] == 'q3']

# Get the baseline (wo) values
baseline_q3 = q3_df[q3_df['execution_mode'] == 'wo'].iloc[0]

# Select the metrics to analyze
q3_metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory", 
    "shuffleTotalBytesRead",
    "shuffleBytesWritten"
]

# Create a dataframe to store improvement ratios
improvements = []

# Calculate improvements for each configuration compared to baseline
for _, row in q3_df[q3_df['execution_mode'] != 'wo'].iterrows():
    for metric in q3_metrics:
        # Calculate improvement ratio (baseline / configuration)
        improvement = baseline_q3[metric] / row[metric] if row[metric] > 0 else 0
        improvements.append({
            'optimization': row['execution_type'],
            'metric': metric,
            'improvement_ratio': improvement
        })

# Convert to DataFrame
improvements_df = pd.DataFrame(improvements)

# Print the improvement values for memory usage
memory_improvements = improvements_df[improvements_df['metric'] == 'peakExecutionMemory']
print("Memory improvement ratios:")
for _, row in memory_improvements.iterrows():
    print(f"{row['optimization']}: {row['improvement_ratio']:.2f}x")

# Define better labels for metrics
metric_labels = {
    "elapsedTime": "Tempo de Execução",
    "executorCpuTime": "Tempo de CPU",
    "peakExecutionMemory": "Uso de Memória",
    "shuffleTotalBytesRead": "Dados de Shuffle Lidos",
    "shuffleBytesWritten": "Dados de Shuffle Escritos"
}

# Map metrics to better labels
improvements_df['metric_label'] = improvements_df['metric'].map(metric_labels)

# Order the metrics in a logical sequence
metric_order = ["Tempo de Execução", "Tempo de CPU", "Uso de Memória", 
                "Dados de Shuffle Lidos", "Dados de Shuffle Escritos"]
improvements_df['metric_order'] = improvements_df['metric_label'].apply(lambda x: metric_order.index(x))
improvements_df = improvements_df.sort_values('metric_order')

# Define a color map for optimization types to ensure consistency
color_map = {
    'Com CBO': '#636efa',
    'Com AQE': '#00cc96',
    'Com CBO+AQE': '#ef553b'
}

# Create the visualization
fig = px.bar(
    improvements_df,
    x='metric_label',
    y='improvement_ratio',
    color='optimization',
    barmode='group',
    text=improvements_df['improvement_ratio'].round(2),  # Add text labels on bars
    title='Ganhos de Desempenho para Q3 sob Diferentes Otimizações',
    labels={
        'improvement_ratio': 'Fator de Melhoria (x vezes)',
        'metric_label': 'Métrica',
        'optimization': 'Otimização'
    },
    category_orders={"metric_label": metric_order},  # Ensure consistent order
    color_discrete_map=color_map  # Use consistent colors
)

# Add a reference line at y=1 (no improvement)
fig.add_hline(y=1, line_dash="dash", line_color="gray")

# Improve layout
fig.update_layout(
    xaxis_title="Métrica",
    yaxis_title="Fator de Melhoria (x vezes)",
    legend_title="Otimização"
)

# Update text position
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

Memory improvement ratios:
Com CBO: 1.00x
Com CBO+AQE: 134.30x
Com AQE: 134.30x


In [23]:
# Filter data for query q19
q19_df = df_all_queries[df_all_queries['query'] == 'q19']

# Get the baseline (wo) values
baseline_q19 = q19_df[q19_df['execution_mode'] == 'wo'].iloc[0]

# Select the metrics to analyze
q19_metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory", 
    "shuffleTotalBytesRead",
    "shuffleBytesWritten"
]

# Create a dataframe to store improvement ratios
improvements = []

# Calculate improvements for each configuration compared to baseline
for _, row in q19_df[q19_df['execution_mode'] != 'wo'].iterrows():
    for metric in q19_metrics:
        # Calculate improvement ratio (baseline / configuration)
        improvement = baseline_q19[metric] / row[metric] if row[metric] > 0 else 0
        improvements.append({
            'optimization': row['execution_type'],
            'metric': metric,
            'improvement_ratio': improvement
        })

# Convert to DataFrame
improvements_df = pd.DataFrame(improvements)

# Print the improvement values for memory usage
memory_improvements = improvements_df[improvements_df['metric'] == 'peakExecutionMemory']
print("Memory improvement ratios:")
for _, row in memory_improvements.iterrows():
    print(f"{row['optimization']}: {row['improvement_ratio']:.2f}x")

# Define better labels for metrics
metric_labels = {
    "elapsedTime": "Tempo de Execução",
    "executorCpuTime": "Tempo de CPU",
    "peakExecutionMemory": "Uso de Memória",
    "shuffleTotalBytesRead": "Dados de Shuffle Lidos",
    "shuffleBytesWritten": "Dados de Shuffle Escritos"
}

# Map metrics to better labels
improvements_df['metric_label'] = improvements_df['metric'].map(metric_labels)

# Order the metrics in a logical sequence
metric_order = ["Tempo de Execução", "Tempo de CPU", "Uso de Memória", 
                "Dados de Shuffle Lidos", "Dados de Shuffle Escritos"]
improvements_df['metric_order'] = improvements_df['metric_label'].apply(lambda x: metric_order.index(x))
improvements_df = improvements_df.sort_values('metric_order')

# Define a color map for optimization types to ensure consistency
color_map = {
    'Com CBO': '#636efa',
    'Com AQE': '#00cc96',
    'Com CBO+AQE': '#ef553b'
}

# Create the visualization
fig = px.bar(
    improvements_df,
    x='metric_label',
    y='improvement_ratio',
    color='optimization',
    barmode='group',
    text=improvements_df['improvement_ratio'].round(2),  # Add text labels on bars
    title='Ganhos de Desempenho para q19 sob Diferentes Otimizações',
    labels={
        'improvement_ratio': 'Fator de Melhoria (x vezes)',
        'metric_label': 'Métrica',
        'optimization': 'Otimização'
    },
    category_orders={"metric_label": metric_order},  # Ensure consistent order
    color_discrete_map=color_map  # Use consistent colors
)

# Add a reference line at y=1 (no improvement)
fig.add_hline(y=1, line_dash="dash", line_color="gray")

# Improve layout
fig.update_layout(
    xaxis_title="Métrica",
    yaxis_title="Fator de Melhoria (x vezes)",
    legend_title="Otimização"
)

# Update text position
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

Memory improvement ratios:
Com CBO: 1.00x
Com CBO+AQE: 10.68x
Com AQE: 10.68x


In [26]:
# Filter data for query q55
q55_df = df_all_queries[df_all_queries['query'] == 'q55']

# Get the baseline (wo) values
baseline_q55 = q55_df[q55_df['execution_mode'] == 'wo'].iloc[0]

# Select the metrics to analyze
q55_metrics = [
    "elapsedTime",
    "executorCpuTime",
    "peakExecutionMemory", 
    "shuffleTotalBytesRead",
    "shuffleBytesWritten"
]

# Create a dataframe to store improvement ratios
improvements = []

# Calculate improvements for each configuration compared to baseline
for _, row in q55_df[q55_df['execution_mode'] != 'wo'].iterrows():
    for metric in q55_metrics:
        # Calculate improvement ratio (baseline / configuration)
        improvement = baseline_q55[metric] / row[metric] if row[metric] > 0 else 0
        improvements.append({
            'optimization': row['execution_type'],
            'metric': metric,
            'improvement_ratio': improvement
        })

# Convert to DataFrame
improvements_df = pd.DataFrame(improvements)

# Print the improvement values for memory usage
memory_improvements = improvements_df[improvements_df['metric'] == 'peakExecutionMemory']
print("Memory improvement ratios:")
for _, row in memory_improvements.iterrows():
    print(f"{row['optimization']}: {row['improvement_ratio']:.2f}x")

# Define better labels for metrics
metric_labels = {
    "elapsedTime": "Tempo de Execução",
    "executorCpuTime": "Tempo de CPU",
    "peakExecutionMemory": "Uso de Memória",
    "shuffleTotalBytesRead": "Dados de Shuffle Lidos",
    "shuffleBytesWritten": "Dados de Shuffle Escritos"
}

# Map metrics to better labels
improvements_df['metric_label'] = improvements_df['metric'].map(metric_labels)

# Order the metrics in a logical sequence
metric_order = ["Tempo de Execução", "Tempo de CPU", "Uso de Memória", 
                "Dados de Shuffle Lidos", "Dados de Shuffle Escritos"]
improvements_df['metric_order'] = improvements_df['metric_label'].apply(lambda x: metric_order.index(x))
improvements_df = improvements_df.sort_values('metric_order')

# Define a color map for optimization types to ensure consistency
color_map = {
    'Com CBO': '#636efa',
    'Com AQE': '#00cc96',
    'Com CBO+AQE': '#ef553b'
}

# Create the visualization
fig = px.bar(
    improvements_df,
    x='metric_label',
    y='improvement_ratio',
    color='optimization',
    barmode='group',
    text=improvements_df['improvement_ratio'].round(2),  # Add text labels on bars
    title='Ganhos de Desempenho para q55 sob Diferentes Otimizações',
    labels={
        'improvement_ratio': 'Fator de Melhoria (x vezes)',
        'metric_label': 'Métrica',
        'optimization': 'Otimização'
    },
    category_orders={"metric_label": metric_order},  # Ensure consistent order
    color_discrete_map=color_map  # Use consistent colors
)

# Add a reference line at y=1 (no improvement)
fig.add_hline(y=1, line_dash="dash", line_color="gray")

# Improve layout
fig.update_layout(
    xaxis_title="Métrica",
    yaxis_title="Fator de Melhoria (x vezes)",
    legend_title="Otimização"
)

# Update text position
fig.update_traces(textposition='outside')

# Show the plot
fig.show()

Memory improvement ratios:
Com CBO: 1.00x
Com CBO+AQE: 16.04x
Com AQE: 16.04x


In [27]:
# Mapeamento de queries para complexidade estrutural
complexity_mapping = {
    'q42': 'Baixa',
    'q43': 'Baixa',
    'q3': 'Baixa',
    'q52': 'Baixa',
    'q55': 'Baixa',
    'q27': 'Média',
    'q67': 'Média',
    'q19': 'Alta',
    'q98': 'Alta',
    'q73': 'Alta'
}

# Adicionar a complexidade estrutural ao dataframe de speedup
speedup_df['complexity'] = speedup_df['query'].map(complexity_mapping)

# Agrupar por complexidade estrutural e tipo de execução, calculando a média do speedup
aggregated_speedup = speedup_df.groupby(['complexity', 'optimization'])['speedup'].mean().reset_index()

# Criar o gráfico de barras
fig = px.bar(
    aggregated_speedup,
    x='complexity',
    y='speedup',
    color='optimization',
    barmode='group',
    title='Speedup Médio por Complexidade Estrutural e Tipo de Execução',
    labels={
        'complexity': 'Complexidade Estrutural',
        'speedup': 'Speedup Médio (x vezes)',
        'optimization': 'Tipo de Execução'
    },
    text=aggregated_speedup['speedup'].round(2)  # Adicionar rótulos com valores arredondados
)

# Melhorar o layout
fig.update_layout(
    xaxis_title="Complexidade Estrutural",
    yaxis_title="Speedup Médio (x vezes)",
    legend_title="Tipo de Execução"
)

# Mostrar o gráfico
fig.show()

In [28]:
from sklearn.preprocessing import MinMaxScaler

# Criação de gráficos de radar para visualização multidimensional de métricas

# Importar bibliotecas necessárias (já importadas nas células anteriores)
import plotly.graph_objects as go
import plotly.io as pio

# Configurar para gerar imagens de alta qualidade
pio.templates.default = "plotly_white"

# 1. Radar plot para Q42 específico
q42_data = df_all_queries[df_all_queries['query'] == 'q42']

# Normalizar os dados para a escala 0-1
scaler_q42 = MinMaxScaler()
q42_data_norm = pd.DataFrame(
    scaler_q42.fit_transform(q42_data[radar_metrics]), 
    columns=radar_metrics
)
q42_data_norm['execution_type'] = q42_data['execution_type'].values

# Definir ordem específica dos tipos de execução para manter consistência na legenda
execution_order = ["Sem otimizações", "Com CBO", "Com AQE", "Com CBO+AQE"]

# Criar gráfico de radar para Q42
fig_q42 = go.Figure()

# Mapear cores consistentes para os tipos de execução
color_map = {
    'Sem otimizações': '#636EFA',
    'Com CBO': '#EF553B',
    'Com AQE': '#00CC96',
    'Com CBO+AQE': '#AB63FA'
}

# Adicionar cada tipo de execução em ordem específica para controlar a legenda
for exec_type in execution_order:
    if exec_type in q42_data_norm['execution_type'].values:
        exec_data = q42_data_norm[q42_data_norm['execution_type'] == exec_type]
        
        fig_q42.add_trace(go.Scatterpolar(
            r=exec_data[radar_metrics].values[0],
            theta=radar_metrics,
            fill='toself',
            name=exec_type,
            line=dict(color=color_map[exec_type]),
            opacity=0.7
        ))

# Melhorar rótulos das métricas
metric_labels = {
    'elapsedTime': 'Tempo de Execução',
    'executorRunTime': 'Tempo de Executor',
    'executorCpuTime': 'Tempo de CPU',
    'shuffleTotalBytesRead': 'Bytes Lidos (Shuffle)',
    'shuffleBytesWritten': 'Bytes Escritos (Shuffle)',
    'peakExecutionMemory': 'Pico de Memória'
}

# Atualizar layout para o gráfico de Q42
fig_q42.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            tickfont=dict(size=10)
        ),
        angularaxis=dict(
            tickfont=dict(size=10),
            tickangle=0,
            rotation=90,
            direction='counterclockwise',
            tickmode='array',
            tickvals=radar_metrics,
            ticktext=[metric_labels[m] for m in radar_metrics]
        )
    ),
    title='Comparação Multidimensional de Métricas para Query Q42',
    title_font_size=16,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=10)
    ),
    height=600,
    width=700,
    margin=dict(l=80, r=80, t=100, b=100)
)

# Mostrar o gráfico de Q42
fig_q42.show()

# 2. Radar plot com médias gerais
# Reutilizar o código da célula 15, mas com melhorias visuais

# Calcular a média das métricas por tipo de execução
mean_data = df_all_queries.groupby('execution_type')[radar_metrics].mean().reset_index()

# Normalizar os dados para a escala 0-1
scaler_avg = MinMaxScaler()
mean_data_norm = pd.DataFrame(
    scaler_avg.fit_transform(mean_data[radar_metrics]), 
    columns=radar_metrics
)
mean_data_norm['execution_type'] = mean_data['execution_type']

# Criar o gráfico de radar com médias
fig_avg = go.Figure()

# Adicionar cada tipo de execução em ordem específica
for exec_type in execution_order:
    if exec_type in mean_data_norm['execution_type'].values:
        exec_data = mean_data_norm[mean_data_norm['execution_type'] == exec_type]
        
        fig_avg.add_trace(go.Scatterpolar(
            r=exec_data[radar_metrics].values[0],
            theta=radar_metrics,
            fill='toself',
            name=exec_type,
            line=dict(color=color_map[exec_type]),
            opacity=0.7
        ))

# Atualizar layout para o gráfico de médias
fig_avg.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            tickfont=dict(size=10)
        ),
        angularaxis=dict(
            tickfont=dict(size=10),
            tickangle=0,
            rotation=90,
            direction='counterclockwise',
            tickmode='array',
            tickvals=radar_metrics,
            ticktext=[metric_labels[m] for m in radar_metrics]
        )
    ),
    title='Comparação Multidimensional do Impacto Médio dos Otimizadores',
    title_font_size=16,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(size=10)
    ),
    height=600,
    width=700,
    margin=dict(l=80, r=80, t=100, b=100)
)

# Mostrar o gráfico de médias
fig_avg.show()